# EDA of Agoda's flights and hotel dataset using SQL

This notebook analyzes the flight and hotel bookings of Agoda's customers from 26-Sept-2019 to 24-July-2023. The aim of this project is to analyze customer booking behavior and booking pricing by answering the following questions:

- What are the booking trends by gender, age and month?
- What are the factors that affect flight and hotel prices?



## Set Up

The following code connects the notebook to the local MSSQL database, allowing its output to be display in this notebook.

In [1]:
from sqlalchemy import create_engine
import pandas as pd

# Windows Authentication (Trusted_Connection) - Connection to Local MSSQL Server
conn_str = (
    "mssql+pyodbc://DESKTOP-2M488OT\\SQLEXPRESS/agoda"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

engine = create_engine(conn_str)

The flights, hotels, and users tables are created using the following code.

In [2]:
# Load the CSV data into a DataFrame
flights = pd.read_csv("data/flights.csv")
hotels = pd.read_csv("data/hotels.csv")
users = pd.read_csv("data/users.csv")

# Push the DataFrame to the MSSQL database (creating the table)
flights.to_sql("flights", con=engine, if_exists="replace", index=False)
hotels.to_sql("hotels", con=engine, if_exists="replace", index=False)
users.to_sql("users", con=engine, if_exists="replace", index=False)

83

## 1. Booking Trends By Demographic Factors

### Gender

The following query examines the number of flight bookings and the average flight prices by different genders. The results indicate even distribution of bookings among the different genders. The similar average prices allude that spending power or purchasing habit of Agoda's customers is not heavily dependent on gender.

In [3]:
query = """
-- flight bookings contributed by different genders
SELECT gender, COUNT(*) AS number_of_bookings, ROUND(AVG(price),2) AS average_price
FROM flights 
INNER JOIN users 
ON flights.userCode = users.code
GROUP BY gender;
"""

result = pd.read_sql(query, engine)

result

,gender,number_of_bookings,average_price
0,female,91580,956.58
1,male,91248,960.96
2,none,89060,954.51


This query identifies the most popular travel destinations by gender, sorting the results in descending order by the number of bookings. Florianopolis is the top destination across all genders.

In [4]:
query = """
-- popular destinations by gender
SELECT flights.[to], users.gender, COUNT(*) AS bookings
FROM flights
INNER JOIN users
ON flights.userCode = users.code
GROUP BY flights.[to], users.gender
ORDER BY 2,3 DESC;
"""

result = pd.read_sql(query, engine)

result

,to,gender,bookings
0,Florianopolis (SC),female,19567
1,Aracaju (SE),female,12626
2,Campo Grande (MS),female,11050
3,Brasilia (DF),female,10470
4,Recife (PE),female,10260
5,Natal (RN),female,8112
6,Sao Paulo (SP),female,8070
7,Salvador (BH),female,5718
8,Rio de Janeiro (RJ),female,5707
9,Florianopolis (SC),male,19914


### Age Group

The following query examines the number of flight bookings and the average flight prices by age groups. The age group of 20-29 and 60-69 contribute less to the total number of flights booked. Meanwhile, the average flight price does not variable significantly among different age groups. 

In [5]:
query = """
-- flights by age group
SELECT 
    CONCAT(floor(age/10)*10,'-',floor(age/10)*10+9) AS age_group, 
    COUNT(*) AS number_of_bookings, 
    ROUND(AVG(price),2) AS average_price
FROM flights 
INNER JOIN users 
ON flights.userCode = users.code
GROUP BY floor(age/10)*10
ORDER BY 1;
"""

result = pd.read_sql(query, engine)

result

,age_group,number_of_bookings,average_price
0,20-29,52956,963.53
1,30-39,64670,962.19
2,40-49,61548,953.59
3,50-59,57260,957.44
4,60-69,35454,945.87


This query identifies the most popular travel destinations across age groups, sorting the results in descending order by the number of bookings. Florianopolis is the top destination across all age groups.

In [6]:
query = """
-- popular destinations by age group
SELECT 
    flights.[to], 
    CONCAT(floor(age/10)*10,'-',floor(age/10)*10+9) AS age_group, 
    COUNT(*) AS bookings
FROM flights
INNER JOIN users
ON flights.userCode = users.code
GROUP BY flights.[to], floor(age/10)*10
ORDER BY 2,3 DESC;
"""

result = pd.read_sql(query, engine)

result

,to,age_group,bookings
0,Florianopolis (SC),20-29,12180
1,Aracaju (SE),20-29,7050
2,Campo Grande (MS),20-29,6008
3,Brasilia (DF),20-29,5891
4,Recife (PE),20-29,5804
5,Sao Paulo (SP),20-29,4701
6,Natal (RN),20-29,4665
7,Rio de Janeiro (RJ),20-29,3347
8,Salvador (BH),20-29,3310
9,Florianopolis (SC),30-39,14433


## 2. Flight and Hotel Bookings Monthly Revenue Trends

This query calculates the total flights and hotel stays for each month by combining records from flights and hotels.

In [7]:
query = """
-- bookings by months
SELECT a.travel_month, a.flight_bookings, b.hotel_bookings, a.flight_bookings+b.hotel_bookings AS total_bookings
FROM (
	SELECT MONTH(date) AS travel_month, COUNT(*) AS flight_bookings
	FROM flights
	GROUP BY MONTH(date)
	) AS a
INNER JOIN (
	SELECT MONTH(date) AS travel_month, COUNT(*) AS hotel_bookings
	FROM hotels
	GROUP BY MONTH(date)
	) AS b
ON a.travel_month = b.travel_month
ORDER BY 4 DESC;
"""

result = pd.read_sql(query, engine)

result

,travel_month,flight_bookings,hotel_bookings,total_bookings
0,10,28980,4504,33484
1,12,26346,4052,30398
2,11,25764,3634,29398
3,1,25587,3775,29362
4,4,22607,3576,26183
5,3,22741,3380,26121
6,2,22387,3284,25671
7,5,20968,2871,23839
8,7,20113,3079,23192
9,9,19468,3011,22479


This query calculates the total revenue generated each month by combining revenue from flights and hotels.

In [8]:
query = """
-- revenue by month
SELECT a.travel_month, a.flight_revenue, b.hotel_revenue, ROUND(a.flight_revenue + b.hotel_revenue,2) AS total_revenue
FROM (
    SELECT MONTH(date) AS travel_month, ROUND(SUM(price), 2) AS flight_revenue
    FROM flights
    GROUP BY MONTH(date)
) AS a
INNER JOIN (
    SELECT MONTH(date) AS travel_month, ROUND(SUM(total), 2) AS hotel_revenue
    FROM hotels
    GROUP BY MONTH(date)
) AS b
ON a.travel_month = b.travel_month
ORDER BY total_revenue DESC;
"""

result = pd.read_sql(query, engine)

result

,travel_month,flight_revenue,hotel_revenue,total_revenue
0,10,27663040.66,2417426.96,30080467.62
1,12,25197015.56,2182197.21,27379212.77
2,11,24672328.58,1948201.51,26620530.09
3,1,24334009.62,1990376.79,26324386.41
4,3,21820447.99,1817208.18,23637656.17
5,4,21599832.60,1909806.30,23509638.90
6,2,21455153.50,1767709.29,23222862.79
7,5,20185235.75,1543882.23,21729117.98
8,7,19335078.33,1646520.04,20981598.37
9,9,18565907.08,1601906.05,20167813.13


From the results, we can see that
- The months with the highest number of bookings and revenue fall between October and January, which aligns with the holiday season and year-end vacations. 
- This indicates a seasonal peak in travel-related bookings during this period.

## 3. Flight and Hotel Price Factors

### Distance

This query calculates the average price of flights based on the distance traveled, grouped into intervals of 100 kilometers. The distance intervals are displayed as ranges (e.g., 100-199 km, 200-299 km), and for each range, the average price of the flights within that range is calculated.

The results show that, as the distance increases, the average flight price also tends to rise, with flights in the 900-999 km range having the highest average price of 1348.25.

In [9]:
query = """
-- price by distance
SELECT 
	CONCAT(FLOOR(distance/100)*100,'-',FLOOR(distance/100)*100+99) AS flight_distance,
	ROUND(AVG(price),2) AS average_price
FROM flights
GROUP BY FLOOR(distance/100)*100
ORDER BY 1;
"""

result = pd.read_sql(query, engine)

result

,flight_distance,average_price
0,100-199,494.69
1,200-299,673.56
2,300-399,796.14
3,400-499,792.84
4,500-599,961.07
5,600-699,1099.14
6,700-799,1187.10
7,800-899,1274.60
8,900-999,1348.25


### Location

This query shows the average flight price and average hotel price for various destinations.

Insights:
- Florianopolis (SC) and Salvador (BH) have the highest average prices for flights among the destinations listed, at 1185.69 and 1264.35, respectively.
- Campo Grande (MS) has the lowest average hotel price (60.39), while Florianopolis (SC) has the highest hotel price (313.02).
- There seems to be a relationship between higher flight prices and higher hotel prices, as seen in Salvador (BH), which has both high flight and hotel prices.

In [10]:
query = """
-- average prices by destinations
SELECT a.destination, a.avg_flight_price, b.avg_hotel_price
FROM (
	SELECT [to] as destination, ROUND(AVG(price),2) as avg_flight_price
	FROM flights
	GROUP BY [to]
	) AS a
LEFT JOIN (
	SELECT place, ROUND(AVG(price),2) as avg_hotel_price
	FROM hotels
	GROUP BY place
	) AS b
ON a.destination = b.place
"""

result = pd.read_sql(query, engine)

result

,destination,avg_flight_price,avg_hotel_price
0,Salvador (BH),1264.35,263.41
1,Florianopolis (SC),1185.69,313.02
2,Sao Paulo (SP),648.34,139.10
3,Campo Grande (MS),841.35,60.39
4,Brasilia (DF),766.76,247.62
5,Natal (RN),826.87,242.88
6,Rio de Janeiro (RJ),846.77,165.99
7,Recife (PE),951.23,312.83
8,Aracaju (SE),1065.24,208.04


### Month

This query shows the average flight and hotel prices for each month of travel:

Insights:
- The highest average flight price is observed in June (962.74), followed closely by May (962.67).
- The lowest flight prices occur in January (951.03) and September (953.66).
- Hotel prices are generally consistent throughout the year, ranging from a low of 212.38 in January to a high of 215.59 in June.

This data can help travelers identify which months might offer the best deals in terms of both flights and accommodation.

In [11]:
query = """
-- avg price by months
SELECT a.travel_month, a.avg_flight_price, b.avg_hotel_price
FROM (
SELECT MONTH(date) AS travel_month, ROUND(AVG(price),2) AS avg_flight_price
FROM flights
GROUP BY MONTH(date)
    ) AS a
LEFT JOIN (
	SELECT MONTH(date) AS travel_month, ROUND(AVG(price),2) as avg_hotel_price
	FROM hotels
	GROUP BY MONTH(date)
	) AS b
ON a.travel_month = b.travel_month
ORDER BY 2 DESC;
"""

result = pd.read_sql(query, engine)

result

,travel_month,avg_flight_price,avg_hotel_price
0,6,962.74,215.59
1,5,962.67,215.40
2,7,961.32,213.26
3,3,959.52,215.12
4,2,958.38,214.09
5,8,958.25,214.79
6,11,957.63,214.00
7,12,956.39,215.09
8,4,955.45,214.47
9,10,954.56,215.11


### Flight Route

This query provides data on flight bookings, showing the departure city (from), destination city (to), the number of bookings (bookings), and the average price (avg_price) for each route.

Insights:
- Most Booked Route: The highest number of bookings is for the route Sao Paulo (SP) to Florianopolis (SC) with 6717 bookings, at an average price of 1380.88.
- Lowest Average Price: The route with the lowest average price is Florianopolis (SC) to Rio de Janeiro (RJ), with an average price of 474.81, but it still has 5807 bookings, making it a popular and affordable route.
- High Price Routes: Routes like Sao Paulo (SP) to Florianopolis (SC) are more expensive, with prices around 1380.88 but maintain a high number of bookings.

In [12]:
query = """
-- flight bookings and avg price by flight route
SELECT [from], [to], COUNT(*) AS bookings, ROUND(AVG(price),2) AS avg_price
FROM flights
GROUP BY [from], [to]
ORDER BY 4 DESC
"""

result = pd.read_sql(query, engine)

result

,from,to,bookings,avg_price
0,Sao Paulo (SP),Florianopolis (SC),6717,1380.88
1,Campo Grande (MS),Rio de Janeiro (RJ),2415,1371.03
2,Brasilia (DF),Salvador (BH),2009,1366.80
3,Aracaju (SE),Salvador (BH),2918,1364.97
4,Rio de Janeiro (RJ),Recife (PE),1897,1361.20
...,...,...,...,...
65,Sao Paulo (SP),Campo Grande (MS),3597,486.15
66,Aracaju (SE),Natal (RN),3937,484.11
67,Natal (RN),Aracaju (SE),3937,482.38
68,Brasilia (DF),Rio de Janeiro (RJ),1994,481.18


### Agency

This query returns the average flight and hotel prices for different agencies:

Insights:
- CloudFy: Offers a slightly cheaper flight at an average price of 918.90 and a hotel price of 213.73.
- Rainbow: Has a very similar pricing model to CloudFy with an average flight price of 919.78 and hotel price of 214.76.
- FlyingDrops: Charges significantly higher for flights, with an average price of 1186.16, but the hotel price is almost the same as Rainbow at 215.61.

In [13]:
query = """
-- average prices by agency
SELECT f.agency, 
       ROUND(AVG(f.price), 2) AS avg_flight_price, 
       ROUND(AVG(h.price), 2) AS avg_hotel_price
FROM flights f
LEFT JOIN hotels h ON f.travelCode = h.travelCode
GROUP BY f.agency;
"""

result = pd.read_sql(query, engine)

result


,agency,avg_flight_price,avg_hotel_price
0,CloudFy,918.90,213.73
1,Rainbow,919.78,214.76
2,FlyingDrops,1186.16,215.61


### Flight Type

The following query examines the number of flight bookings and the average flight prices by flight type. The average prices are in accordance to business logic, whereby first class tickets record higher prices than premium and economic tickets. Meanwhile, more first class bookings were recorded in this dataset, indicating that first class customers contributed more to the revenue of the company during this time period.

In [14]:
query = """
-- number of booking and average price by flight type
SELECT flightType, COUNT(*) AS number_of_bookings, ROUND(AVG(price),2) AS avg_price
FROM flights
GROUP BY flightType;
"""

result = pd.read_sql(query, engine)

result

,flightType,number_of_bookings,avg_price
0,economic,77466,658.44
1,premium,78004,920.39
2,firstClass,116418,1181.07


## 4. Top Customer

This query returns the names of users along with their corresponding flight, hotel, and total revenue. These are the top 10 customers who have spent the most on flights and hotels combined.

In [15]:
query = """
-- top spenders
SELECT TOP 10 users.name, c.flight_revenue, c.hotel_revenue, c.total_revenue
FROM (
	SELECT a.userCode, 
		a.flight_revenue, 
		ISNULL(b.hotel_revenue, 0) AS hotel_revenue, 
		ROUND(a.flight_revenue+ISNULL(b.hotel_revenue, 0),2) AS total_revenue
	FROM (
		SELECT userCode, ROUND(SUM(price),2) AS flight_revenue
		FROM flights
		GROUP BY userCode
		) AS a
	FULL OUTER JOIN (
		SELECT userCode, ROUND(SUM(total),2) AS hotel_revenue
		FROM hotels
		GROUP BY userCode
		) AS b
	ON a.userCode = b.userCode
	) AS c
INNER JOIN users 
ON c.userCode = users.code
ORDER BY 4 DESC;
"""

result = pd.read_sql(query, engine)

result

,name,flight_revenue,hotel_revenue,total_revenue
0,Tiffany Behm,442901.02,34051.80,476952.82
1,Christopher Mccormick,427202.93,34586.09,461789.02
2,Steven Smith,426133.03,34725.74,460858.77
3,Jessie Armstrong,427218.92,30112.59,457331.51
4,Lyndon Germain,420362.27,34435.88,454798.15
5,Helen Warner,411852.92,40999.86,452852.78
6,Jeffrey Ramage,420462.47,32103.16,452565.63
7,Laurel Rodriguez,423936.24,28170.15,452106.39
8,Albert Garroutte,420398.06,30218.54,450616.60
9,Trevor Robinson,418347.29,30776.36,449123.65
